In [1]:
from senticAPIextractor import extractPolarity
from senticAPIextractor import extractAspects

from datetime import datetime, timedelta

import urllib.parse

import json

import numpy as np
import pandas as pd
import string
printable = set(string.printable)

In [2]:
tickers = ['AAPL', 'ADBE', 'ADI', 'ADP', 'ADSK', 'AKAM', 'ALGN',
          'ALXN', 'AMAT', 'AMGN', 'AMZN', 'ATVI', 'AVGO', 'BIDU', 
          'BIIB', 'BMRN', 'CA', 'CELG', 'CERN', 'CHTR', 'CMCSA',
          'COST', 'CSCO', 'CSX', 'CTAS', 'CTRP', 'CTSH', 'CTXS',
          'DISCA', 'DISH', 'DLTR', 'EA', 'EBAY', 'ESRX', 'EXPE',
          'FAST', 'FB', 'FISV', 'FOXA', 'GILD', 'GOOGL', 'HAS',
          'HOLX', 'HSIC', 'IDXX', 'ILMN', 'INCY', 'INTC', 'INTU',
          'ISRG', 'JBHT', 'JD', 'KHC', 'KLAC', 'LBTYA', 'LRCX', 'MAR',
          'MAT', 'MCHP', 'MDLZ', 'MELI', 'MNST', 'MSFT', 'MU', 'MXIM',
          'MYL', 'NFLX', 'NTES', 'NVDA', 'ORLY', 'PAYX', 'PCAR', 'PCLN',
          'PYPL', 'QCOM', 'REGN', 'ROST', 'SBUX', 'SHPG', 'SIRI', 'STX',
          'SWKS', 'SYMC', 'TMUS', 'TSCO', 'TSLA', 'TXN', 'ULTA', 'VIAB',
          'VOD', 'VRSK', 'VRTX', 'WBA', 'WYNN', 'XLNX', 'XRAY']

In [ ]:
for ticker in tickers:
    print(ticker)
    news =  pd.read_csv('/home/simone/Desktop/NLFF/intrinioDataset/preprocessing/preprocessed/'+ticker+'.csv')
    news.drop_duplicates(subset=['PUBLICATION_DATE'], inplace=True)
    news = news.sort_values(by=['PUBLICATION_DATE'])
    news = news.reset_index(drop=True)
    news = news.drop(['Unnamed: 0'], axis=1)
    notNaN = news['SUMMARY'].dropna(how='any').index.tolist()
    NaN = []
    for i in news.index:
        if(i not in notNaN):
            NaN.append(i)
    news = news.drop(NaN)
    # #news.head(10)
    news = news.reset_index(drop=True)
    news['PUBLICATION_DATE'] = [datetime.strptime(row, '%Y-%m-%d %H:%M:%S +%f') for row in news['PUBLICATION_DATE']]
    
    
    timeSpan = pd.read_csv("/home/simone/Desktop/NLFF/DataSetIndexes/indexes"+ticker+".csv")
    timeSpan.rename(columns={'Unnamed: 0':'date'}, inplace= True)
    timeSpan = timeSpan['date'].tolist()
    timeSpan = [datetime.strptime(row, '%Y-%m-%d %H:%M:%S') for row in timeSpan]


    sentiment = pd.DataFrame(columns=['initTime','NEGATIVE','POSITIVE'])


    #Prendiamo intervallo massimo dove ci sono entrambi i dataset:
    #massimo della data iniziale tra i due
    #minimo della data finale tra i due
    initDate = max(timeSpan[0], news['PUBLICATION_DATE'][0])
    finalDate = min(timeSpan[len(timeSpan)-1], news['PUBLICATION_DATE'][len(news)-1])




    i = 0;
    j = 0
    normal_sum = {'NEGATIVE': 0, 'POSITIVE': 0}

    # ALLINEAMENTO INIZIO
    while(timeSpan[j] < initDate):
        j+=1
    while(news['PUBLICATION_DATE'][i] <= initDate):
        i+=1

    print("Start: "+str(initDate) +" i: "+str(i)+" j: "+str(j))
    print("End: "+str(finalDate))

    #ALLINEAMENTO FINE DENTRO I WHILE
    while( news['PUBLICATION_DATE'][i] < finalDate and timeSpan[j] < finalDate ):
        num_sentiment = 0
        initTime = timeSpan[j]    
        while(i<len(news)-1 and timeSpan[j] > news['PUBLICATION_DATE'][i]):
            if not (timeSpan[j] > news['PUBLICATION_DATE'][i] and timeSpan[j-1] <= news['PUBLICATION_DATE'][i]):
                print("timeSpan["+str(j)+"]: "+str(timeSpan[j])+" news["+str(i)+"] : " +str(news['PUBLICATION_DATE'][i]) + " timeSpan["+str(j-1)+"]: "+str(timeSpan[j-1]))
                assert False
            try:
                sent = extractPolarity(urllib.parse.quote(news['SUMMARY'][i]))['polarity']
                if(sent == 'positive'):
                    normal_sum['POSITIVE']+=1
                if(sent == 'negative'):
                    normal_sum['NEGATIVE']+=1
                num_sentiment +=1
            except:
                print("ERROR news "+str(i))
            if(i%100==0):
                print(i)
            i+=1
        for key, value in normal_sum.items():
            if(num_sentiment != 0):
                normal_sum[key] /=num_sentiment
        j+=1
        sentiment.loc[j] = {'initTime':initTime, 'NEGATIVE': normal_sum['NEGATIVE'], 'POSITIVE': normal_sum['POSITIVE']}

    sentiment.to_csv('SentimentNews/'+ticker+'.csv')

AAPL
Start: 2017-10-21 16:50:00 i: 1 j: 2417
End: 2018-03-27 22:22:02
ERROR news 6
100
ERROR news 113
ERROR news 198
200
ERROR news 287
ERROR news 293
300
ERROR news 328
400
500
ERROR news 513
ERROR news 552
600
ERROR news 648
700
ERROR news 727
ERROR news 736
ERROR news 779
800
ERROR news 849
900
1000
ERROR news 1051
ERROR news 1063
1100
ERROR news 1121
ERROR news 1122
ERROR news 1156
1200
ERROR news 1211
ERROR news 1222
ERROR news 1238
ERROR news 1242
ERROR news 1259
ERROR news 1263
1300
1400
1500
ERROR news 1511
ERROR news 1528
ERROR news 1530
ERROR news 1544
ERROR news 1551
ERROR news 1568
1600
ERROR news 1630
ERROR news 1639
ERROR news 1660
1700
ERROR news 1736
ERROR news 1783
1800
1900
2000
ERROR news 2031
ERROR news 2065
2100
ERROR news 2113
ERROR news 2187
2200
ERROR news 2284
2300
ERROR news 2301
ERROR news 2349
ERROR news 2355
2400
2500
2600
2700
ERROR news 2763
ERROR news 2779
ERROR news 2783
ERROR news 2792
ERROR news 2794
2800
ERROR news 2845
2900
ERROR news 2928
ERROR new

ERROR news 513
ERROR news 520
ERROR news 523
ERROR news 530
ERROR news 535
ERROR news 540
ERROR news 542
ERROR news 543
ERROR news 545
ERROR news 547
ERROR news 552
ERROR news 554
ERROR news 555
ERROR news 559
ERROR news 560
ERROR news 562
ERROR news 563
ERROR news 565
ERROR news 570
ERROR news 574
ERROR news 575
ERROR news 577
ERROR news 579
ERROR news 584
ERROR news 586
ERROR news 588
ERROR news 590
ERROR news 593
ERROR news 595
600
ERROR news 609
ERROR news 614
ERROR news 642
ERROR news 647
ERROR news 649
ERROR news 650
ERROR news 652
ERROR news 668
ERROR news 669
ERROR news 674
ERROR news 677
ERROR news 680
ERROR news 685
700
ERROR news 741
ERROR news 748
ERROR news 764
ERROR news 791
ERROR news 797
800
ERROR news 801
ERROR news 804
ERROR news 806
ALGN
Start: 2017-03-13 15:00:00 i: 261 j: 0
End: 2018-03-07 22:02:00
ERROR news 291
300
ERROR news 301
ERROR news 318
ERROR news 335
ERROR news 391
400
ERROR news 435
ERROR news 445
ERROR news 450
ERROR news 462
500
ERROR news 585
ERROR n

In [ ]:
s = news['SUMMARY'][6]
print(s)
print(extractPolarity(s))
s = urllib.parse.quote(news['SUMMARY'][6])

print(s)
extractPolarity(s)
